# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [2]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import pi
from scipy.fftpack import fft
from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2023-05-17 21:08:28,096 | INFO     |: Logger demo_notebook.log initialized.


In [3]:
# Start the plotter
START_PLOTTER()

Failed to create wl_display (Permission denied)
qt.qpa.plugin: Could not load the Qt platform plugin "wayland" in "" even though it was found.
QStandardPaths: runtime directory '/mnt/wslg/runtime-dir' is not owned by UID 1001, but a directory permissions 0700 owned by UID 1000 GID 1000


In [4]:
STOP_PLOTTER()

2023-05-17 21:08:46,655 | INFO     |: Plotter is stopped


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [92]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-05-16 20:54:07,952 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:e0:6d:32:3c
2023-05-16 20:54:09,985 | INFO     |: Connected to c0:83:e0:6d:32:3c


In [111]:
new  = []
d1   = []
index = []
global datasent

# datasent = 0
def lab11_handler(_, value):
    global new
    global d1
    global index
    new = ble.bytearray_to_string(value)
    index.append(new[:new.find(',')])
    d1.append(new[new.find(',')+1:])

In [112]:
ble.start_notify(ble.uuid["RX_STRING"], lab11_handler)
ble.send_command(CMD.LAB11_SPIN,"0.1")

['0']
['0', '1']
['0', '1', '2']
['0', '1', '2', '3']
['0', '1', '2', '3', '4']
['0', '1', '2', '3', '4', '5']
['0', '1', '2', '3', '4', '5', '6']
['0', '1', '2', '3', '4', '5', '6', '7']
['0', '1', '2', '3', '4', '5', '6', '7', '8']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']


In [113]:
d1    = [ int(x) for x in d1 ]
index = [ int(x) for x in index ]
angle = np.radians(index * 20)

# BELOW IS FROM RYAN CHAN
measured_values = np.array([np.zeros(18),np.zeros(18)])
for i in index:
    measured_values[0][i] = d1[i]/1000
    measured_values[1][i] = np.radians(i * 20)

In [114]:
sensor_ranges = np.array(measured_values[0])[np.newaxis]
sensor_bearings = np.array(measured_values[1])[np.newaxis]

In [104]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        # raise NotImplementedError("get_pose is not implemented")
    
        #from robot.py
        current_odom = self.cmdr.get_pose()
        return self.get_pose_in_degrees(current_odom)

    def perform_observation_loop(self, rot_vel=120):

        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        return(sensor_ranges.T, sensor_bearings.T)
        


In [109]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2023-05-16 20:57:54,797 | INFO     |:  | Number of observations per grid cell: 18
2023-05-16 20:57:54,801 | INFO     |:  | Precaching Views...
2023-05-16 20:58:02,909 | INFO     |:  | Precaching Time: 8.106 secs
2023-05-16 20:58:02,913 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-16 20:58:02,917 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [115]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

print("after grid beliefs")

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

print("obs data")


# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_gt(5 * .3048, 3 * .3048)

# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-16 20:59:35,216 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-16 20:59:35,219 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
after grid beliefs
obs data
2023-05-16 20:59:35,223 | INFO     |: Update Step
2023-05-16 20:59:35,231 | INFO     |:      | Update Time: 0.006 secs
2023-05-16 20:59:35,235 | INFO     |: Bel index     : (10, 6, 9) with prob = 1.0
2023-05-16 20:59:35,237 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-05-16 20:59:35,240 | INFO     |: Belief        : (1.524, 0.610, 10.000)
2023-05-16 21:00:07,623 | INFO     |: Disconnected from c0:83:e0:6d:32:3c


For 5n3, started the robot facing left instead of forward